# 506 group project

This script is completed by Xinyang Qi.

Firstly, import package and read data.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xport
from patsy import dmatrices
from patsy import dmatrix
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [219]:
data1=pd.read_csv('ALQ_D.csv')
data2=pd.read_csv('PAQ_D.csv')
data3=pd.read_csv('PAQIAF_D.csv')
data4=pd.read_csv('SLQ_D.csv')
data5=pd.read_csv('DR1TOT_D.csv')
data6=pd.read_csv('DEMO_D.csv')

Then, extract and clean data.

In [220]:
data2=data2[['SEQN','PAQ520']]
data4=data4[['SEQN','SLD010H']]
data3=data3[['SEQN','PADTIMES','PADDURAT']]
data5=data5[['SEQN','DR1TKCAL','DR1TSUGR','DR1TCAFF']]
data6=data6[['SEQN','RIAGENDR','RIDAGEYR']]

In [221]:
data=pd.merge(data2,data3,on='SEQN',how='inner')
data=pd.merge(data,data4,on='SEQN',how='inner')
data=pd.merge(data,data5,on='SEQN',how='inner')
data=pd.merge(data,data6,on='SEQN',how='inner')

In [222]:
data=data[data.PAQ520 != 9]
data=data[data.PAQ520 != 7]
data=data[data.SLD010H != 77]
data=data[data.SLD010H != 99]
data=data.dropna()
data=data.groupby('SEQN').mean()
data.columns=['act_level','act_times','act_durat','sleep_time','energy','sugars','caffeine','gender','age']

transform 'gender' and 'act_level' to factor variable.

In [225]:
data['gender']=data['gender'].apply(str)
data['act_level']=data['act_level'].apply(str)
data1 = data._get_numeric_data()

In order to improve the model, we select variables based on their vif and Covariance matrix at first. Then, using backward elimination to select variables.

In [227]:
# get y and X dataframes based on this regression:
y, X = dmatrices('sleep_time ~ energy + sugars + caffeine + age + act_times + act_durat -1', data1, return_type='dataframe')

In [228]:
# Calculate VIF
vif=pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
vif["features"]=X.columns

In [229]:
print(vif)

   VIF Factor   features
0    8.491139     energy
1    6.235878     sugars
2    1.631222   caffeine
3    3.287484        age
4    1.901926  act_times
5    2.050207  act_durat


In [230]:
# Calculate covariance matrix
data1.drop('sleep_time',axis=1).corr()

,act_times,act_durat,energy,sugars,caffeine,age
act_times,1.000000,-0.124854,-0.057104,-0.018924,-0.041066,0.138508
act_durat,-0.124854,1.000000,0.119376,0.052507,0.065380,-0.048322
energy,-0.057104,0.119376,1.000000,0.674932,0.110267,-0.199687
sugars,-0.018924,0.052507,0.674932,1.000000,0.039253,-0.210156
caffeine,-0.041066,0.065380,0.110267,0.039253,1.000000,0.206272
age,0.138508,-0.048322,-0.199687,-0.210156,0.206272,1.000000


Create an OLS regression model based on the latest data.

In [231]:
mod = smf.ols(formula='sleep_time ~ act_times + act_durat + caffeine + age + gender', data=data)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             sleep_time   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     10.86
Date:                Wed, 04 Dec 2019   Prob (F-statistic):           2.18e-10
Time:                        11:28:10   Log-Likelihood:                -6164.6
No. Observations:                3605   AIC:                         1.234e+04
Df Residuals:                    3599   BIC:                         1.238e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.1300      0.067    106.390

From the summary, we can find that the predictor of act_durat is not significant. However, maybe it doesn't have linear relationship with sleep time. So we decide to use spline regression to try to improve its fit.

In [235]:
transformed_x1 = dmatrix("bs(data.act_durat, df=3, degree = 3, include_intercept=False)",
                        {"data.act_durat": data.act_durat}, return_type='dataframe')

In [241]:
mod2 = sm.GLM(data.sleep_time, transformed_x1).fit()
print(mod2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             sleep_time   No. Observations:                 3605
Model:                            GLM   Df Residuals:                     3601
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          1.8158
Method:                          IRLS   Log-Likelihood:                -6188.5
Date:                Wed, 04 Dec 2019   Deviance:                       6538.6
Time:                        11:47:31   Pearson chi2:                 6.54e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

Based on the summary above, the act_time^3 is nearly significant at the level of 0.1. So splines may improve our model. We plan to focus on this part in the future.